In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime

# Task 2.1: Preprocessing
# Load datasets
train_df = pd.read_csv("../data/rossmann_store_sales/train.csv", low_memory=False)
store_df = pd.read_csv("../data/rossmann_store_sales/store.csv")
sample_submission_df = pd.read_csv("../data/rossmann_store_sales/sample_submission.csv")

In [ ]:
# Merge store data with train data
train_df = train_df.merge(store_df, on='Store', how='left')

# Prepare time series data for LSTM model
scaler_ts = MinMaxScaler()
ts_data = train_df[['Date', 'Sales']].set_index('Date').resample('D').sum().fillna(0)
ts_data['Sales'] = scaler_ts.fit_transform(ts_data[['Sales']])
